At now the data is item-customer-centric and I need a customer-centric data to solve the customer segmentation problem, and that means to transform the actual dataset to a new one where each row represents a unique customer whose attributes are listed in other columns.

Another challenge is, the description is the only indicator of a product. To solve this problem, I'll cluster product descriptions into a few groups (4 or 5). The values of each one will indicate the total value of the items a customer bought that fit into the respective group.

# Step 1: Basic cleaning

In [112]:
import pandas as pd

data = pd.read_csv('./data/data.csv', encoding='ISO-8859-1')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [113]:
# Looking at how many unique customers there are
data['CustomerID'].nunique()

4372

In [114]:
# And how many customer rows have a NA value
data['CustomerID'].isna().value_counts()

False    406829
True     135080
Name: CustomerID, dtype: int64

In [115]:
# Due to this column is the most important and to fill all the missing values could be inaccurate I decide to drop all rows that have NA values
data = data.dropna()

print(data.shape)
data.head()

(406829, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [116]:
# For this exercise the InvoiceNo and StockCode columns are irrelevant
data.drop(['InvoiceNo','StockCode'],axis=1,inplace=True)
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [117]:
# To prepare the Description column for vectorizing, I'll need to make the words lowercase and remove any punctuaction
data['Description'] = data['Description'].str.replace(r'[^\w\s]','',regex=True).str.lower()
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [118]:
data = data.reset_index().drop('index',axis=1)

In [119]:
# In the future I'll need to know the total amount of monet spent on a product
data['Total'] = data['UnitPrice'] * data['Quantity']

# Step 2: Vectorize the Description

Given the nature of these descriptions, it's probably best to use bag of words instead of TD-IDF. It seems that every word in the description is a keyword of sorts, so vectorizing it simply with bag of words/count is the best way to go.

In [120]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

In [121]:
descriptions = vectorizer.fit_transform(data['Description'])
descriptions.get_shape()

(406829, 2093)

# Step 3: Cluster the Products and Create Features

In [122]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
kmeans.fit(descriptions)

KMeans(n_clusters=5)

In [123]:
data['Product'] = kmeans.labels_
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,Product
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,1
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,2
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,1


### And now it's time to create the customer-centric data

In [124]:
import numpy as np
custom1 = data.pivot_table(values='Total',index='CustomerID', columns='Product', aggfunc=np.sum)
custom1.head()

Product,0,1,2,3,4
CustomerID,,,,,
12346.0,NaN,NaN,0.00,NaN,NaN
12347.0,153.90,145.44,2962.26,561.00,487.40
12348.0,191.00,NaN,1500.88,63.60,41.76
12349.0,244.87,127.95,1041.44,101.49,241.80
12350.0,NaN,NaN,279.20,19.80,35.40


In [125]:
custom1.fillna(value=0, inplace=True)
custom1.head()

Product,0,1,2,3,4
CustomerID,,,,,
12346.0,0.00,0.00,0.00,0.00,0.00
12347.0,153.90,145.44,2962.26,561.00,487.40
12348.0,191.00,0.00,1500.88,63.60,41.76
12349.0,244.87,127.95,1041.44,101.49,241.80
12350.0,0.00,0.00,279.20,19.80,35.40


In [126]:
customers = pd.DataFrame({'CustomerID' : data['CustomerID'].unique()})
customers.sort_values(by='CustomerID',inplace=True)
customers.reset_index(drop=True,inplace=True)
customers.head()

,CustomerID
0,12346.0
1,12347.0
2,12348.0
3,12349.0
4,12350.0


In [127]:
customers = customers.assign(
    Products0 = custom1[0].values,
    Products1 = custom1[1].values,
    Products2 = custom1[2].values,
    Products3 = custom1[3].values,
    Products4 = custom1[4].values
)
customers.head()

,CustomerID,Products0,Products1,Products2,Products3,Products4
0,12346.0,0.00,0.00,0.00,0.00,0.00
1,12347.0,153.90,145.44,2962.26,561.00,487.40
2,12348.0,191.00,0.00,1500.88,63.60,41.76
3,12349.0,244.87,127.95,1041.44,101.49,241.80
4,12350.0,0.00,0.00,279.20,19.80,35.40


# Step 4: Creating Purchasing History Descriptions

In [128]:
custom2 = data.groupby(by='CustomerID')['Total'].agg(['count','min','max','mean','sum'])
custom2.head()

,count,min,max,mean,sum
CustomerID,,,,,
12346.0,2,-77183.60,77183.6,0.000000,0.00
12347.0,182,5.04,249.6,23.681319,4310.00
12348.0,31,13.20,240.0,57.975484,1797.24
12349.0,73,6.64,300.0,24.076027,1757.55
12350.0,17,8.50,40.0,19.670588,334.40


In [129]:
customers = customers.assign(
    PurchaseCount = custom2['count'].values,
    PurchaseMin = custom2['min'].values,
    PurchaseMax = custom2['max'].values,
    PurchaseMean = custom2['mean'].values,
    PurchaseSum = custom2['sum'].values
)
customers.head()

,CustomerID,Products0,Products1,Products2,Products3,Products4,PurchaseCount,PurchaseMin,PurchaseMax,PurchaseMean,PurchaseSum
0,12346.0,0.00,0.00,0.00,0.00,0.00,2,-77183.60,77183.6,0.000000,0.00
1,12347.0,153.90,145.44,2962.26,561.00,487.40,182,5.04,249.6,23.681319,4310.00
2,12348.0,191.00,0.00,1500.88,63.60,41.76,31,13.20,240.0,57.975484,1797.24
3,12349.0,244.87,127.95,1041.44,101.49,241.80,73,6.64,300.0,24.076027,1757.55
4,12350.0,0.00,0.00,279.20,19.80,35.40,17,8.50,40.0,19.670588,334.40


# Step 5: Creating a binary 'foreign' faeture

In [130]:
foreign_c = -data['Country'].isin(['United Kingdom'])
foreign_c = foreign_c.astype('int')

data['Foreign'] = foreign_c
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,Product,Foreign
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,1,0
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2,0
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,2,0
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2,0
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,1,0


In [131]:
customers['Foreign'] = data.groupby('CustomerID')['Foreign'].mean().astype('int').values
customers.head()

,CustomerID,Products0,Products1,Products2,Products3,Products4,PurchaseCount,PurchaseMin,PurchaseMax,PurchaseMean,PurchaseSum,Foreign
0,12346.0,0.00,0.00,0.00,0.00,0.00,2,-77183.60,77183.6,0.000000,0.00,0
1,12347.0,153.90,145.44,2962.26,561.00,487.40,182,5.04,249.6,23.681319,4310.00,1
2,12348.0,191.00,0.00,1500.88,63.60,41.76,31,13.20,240.0,57.975484,1797.24,1
3,12349.0,244.87,127.95,1041.44,101.49,241.80,73,6.64,300.0,24.076027,1757.55,1
4,12350.0,0.00,0.00,279.20,19.80,35.40,17,8.50,40.0,19.670588,334.40,1


In [132]:
customers['Foreign'].value_counts()

0    3950
1     422
Name: Foreign, dtype: int64

In [134]:
data.to_csv('./data/new_data.csv',index=False)
customers.to_csv('./data/customers.csv',index=False)